In [1]:
from datasets import FirstTokenRepeatedImmediately
import pandas as pd
import torch
import numpy as np

/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = FirstTokenRepeatedImmediately(num_points=1000)

In [3]:
# 0th index is data, 1xt index is label
data.train_data.dataset.tensors[0][:10], data.train_data.dataset.tensors[1][:10]

(tensor([[ 6, 16, 10,  3,  7, 10,  5,  8, 18,  9],
         [ 1,  1, 17,  3,  7,  2, 15, 10, 18, 15],
         [ 6,  6, 19,  2, 18,  3, 16, 12, 19,  7],
         [ 7,  7, 17, 13,  4, 11,  5, 19,  6, 16],
         [10,  7, 17, 11, 10, 15, 16,  8,  7, 11],
         [19, 19, 13, 13, 13,  6,  3,  7, 17,  3],
         [19, 19,  7, 14, 13,  6,  1,  8,  7,  7],
         [ 4,  6, 13,  1,  5,  7, 19, 17, 18, 14],
         [14, 14, 13, 19, 12, 17, 19, 13,  9,  8],
         [ 2,  2, 15,  1,  9,  3,  5,  4,  4, 19]]),
 tensor([0, 1, 1, 1, 0, 1, 1, 0, 1, 1]))

In [4]:
def view_class_balance(dataset):
    return pd.Series(next(iter(torch.utils.data.DataLoader(dataset, batch_size=len(data.train_data))))[1]).value_counts()

print("Train data:\n", view_class_balance(data.train_data))
print("Val data:\n", view_class_balance(data.val_data))
print("Test data:\n", view_class_balance(data.test_data))

Train data:
 1    352
0    348
dtype: int64
Val data:
 0    51
1    49
dtype: int64
Test data:
 0    101
1     99
dtype: int64


In [5]:
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
from transformers import BertConfig, BfBertForSequenceClassification
config = BertConfig.from_pretrained(pretrained_model_name_or_path="../../brunoflow/models/bert/config-toy.json")
m = BfBertForSequenceClassification(config=config)


env: XLA_PYTHON_CLIENT_PREALLOCATE=false


2023-03-24 02:08:06.953449: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
/home/kevin/code/rycolab/transformers/src/transformers/models/bert/modeling_bf_bert.py:181: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in zeros is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  bf.Parameter(jnp.zeros(self.position_ids.shape, dtype=jnp.int64), name="position_ids"),


In [6]:
np.log(2)

0.6931471805599453

In [7]:
import brunoflow as bf
from jax import numpy as jnp 
x, y = next(iter(torch.utils.data.DataLoader(data.train_data, batch_size=128)))
# x
outputs = m(bf.Node(jnp.array(x.numpy())), labels=bf.Node(jnp.array(y.numpy())))
outputs.loss, outputs.logits

(node(name: /, val: 0.6931401491165161, grad: 0.0),
 node(name: +, val: [[-7.8270656e-05  1.2060751e-03]
  [-5.3731253e-04  3.3933457e-04]
  [-1.6323072e-04  9.3649223e-04]
  [-5.3741725e-04  3.3912671e-04]
  [ 2.6346208e-04  1.3277408e-03]
  [-7.5162004e-04 -1.9771922e-04]
  [-5.3732458e-04  3.3931778e-04]
  [ 2.6340602e-04  1.3277361e-03]
  [-1.3696360e-04  5.5655290e-04]
  [-6.0457771e-04  2.0210618e-04]
  [-1.6325769e-04  9.3639409e-04]
  [-2.8755754e-04  8.4810890e-04]
  [-3.8415895e-04  1.8672130e-04]
  [-2.8746997e-04  8.4839453e-04]
  [-5.3735333e-04  3.3926807e-04]
  [-3.8423581e-04  1.8644333e-04]
  [-2.2824209e-04  7.3675101e-04]
  [ 2.6349310e-04  1.3277391e-03]
  [ 2.3014363e-04  1.2702040e-03]
  [-6.0173997e-04  3.0150215e-05]
  [-1.6323398e-04  9.3644852e-04]
  [-9.5589145e-05  1.1383181e-03]
  [-9.5576383e-05  1.1382358e-03]
  [-6.0175773e-04  3.0139781e-05]
  [-7.5159845e-04 -1.9766543e-04]
  [-6.0185173e-04  2.9989415e-05]
  [-1.6322579e-04  9.3647896e-04]
  [-2.60118